In [1]:
import sonnet as snt

In [2]:
z = snt.Linear(output_size = 10 )

In [3]:
z

In [4]:
import tensorflow as tf

In [7]:
x = tf.Variable([ [1,2,3] , [2,3,4] ],dtype=tf.float32)

In [8]:
z(x)

<tf.Tensor 'linear_1/add:0' shape=(2, 10) dtype=float32>

In [9]:
y = tf.Variable([ [1,2,3] , [2,3,4] , [4,5,6] ],dtype=tf.float32)

In [11]:
z(y)

<tf.Tensor 'linear_2/add:0' shape=(3, 10) dtype=float32>

In [12]:
zz = tf.Variable([ [1,2,3,4] , [2,3,4,5] , [4,5,6,7] ],dtype=tf.float32)

In [14]:
import collections

In [15]:
DNCState = collections.namedtuple('DNCState', ('access_output', 'access_state','controller_state'))

In [16]:
a = DNCState(10,20,30)

In [17]:
a

DNCState(access_output=10, access_state=20, controller_state=30)

In [22]:
class DNC:
    def __init__(self, nhid=64, nn_output_size = 64, nlayer=1, controller_class = None, 
                 input_size = 10, output_size = 10, mem_slot = 256, 
                 mem_size = 64, read_heads = 4, batch_size = 1, use_cuda=True):
        print( locals() )
        print( self )
        self.__dict__.update(locals())
        print( self.self )
        print( apply_dict( locals() ) )

In [23]:
a = DNC()

{'use_cuda': True, 'batch_size': 1, 'read_heads': 4, 'mem_size': 64, 'mem_slot': 256, 'output_size': 10, 'input_size': 10, 'controller_class': None, 'nlayer': 1, 'nn_output_size': 64, 'nhid': 64, 'self': <__main__.DNC object at 0x7fca29fd8cc0>}


NameError: name 'apply_dict' is not defined

In [3]:
a

In [4]:
a.self

In [10]:
a.self.self.self

In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [25]:
def cvt_coord(i):
    return [(i/5-2)/2., (i%5-2)/2.]

In [27]:
np_coord_tensor = np.zeros((1, 25, 2))
for i in range(25):
    np_coord_tensor[:,i,:] = np.array( cvt_coord(i) )


In [35]:
z = torch.rand( (3,4,5,1,1) )

In [37]:
z.squeeze(dim=4).size()

torch.Size([3, 4, 5, 1])

In [34]:
z.sum(1)


 2.1452  1.7241  2.4386  1.7250  2.0712
 2.2904  2.2635  2.0275  2.8643  1.8148
 2.3643  1.9916  1.5965  1.9353  1.0075
[torch.FloatTensor of size 3x5]

In [1]:
import numpy as np

In [2]:
import json as js

In [4]:
json_file_dev=open('train-v1.1.json').read()

In [5]:
data_dev=js.loads(json_file_dev)

In [6]:
type( data_dev )

dict

In [12]:
data_dev['data']

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [23]:
data_dev['data'][0]['paragraphs'][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'id': '5733be284776f41900661182',
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [31]:
for i,_ in enumerate( data_dev['data'] ):
    print(_)
    print()
    print()
    if i==2:
        break

{'title': 'University_of_Notre_Dame', 'paragraphs': [{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'qas': [{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}, {'answers': [{'answer_start': 

In [32]:
len( data_dev['data'])

442

In [42]:
data_dev['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']

'Saint Bernadette Soubirous'

In [64]:
def prepSQuAD():
    import json
    from nltk.tokenize import word_tokenize
    count = 0
    filenames = ['train']
    for filename in filenames:
        fpr = open("train-v1.1.json", 'r')
        body = fpr.read()
        js = json.loads(body)
        fpw = open(filename+".txt", 'w')
        for c in js["data"]:
            for p in c["paragraphs"]:
                context = p["context"].split(' ')
                context_char = list(p["context"])
                context_pos = {}
                for qa in p["qas"]:

                    question = word_tokenize(qa["question"])

                    if filename == 'train':
                        for a in qa['answers']:
                            answer = a['text'].strip()
                            answer_start = int(a['answer_start'])

                        #add '.' here, just because NLTK is not good enough in some cases
                        answer_words = word_tokenize(answer+'.')
                        if answer_words[-1] == '.':
                            answer_words = answer_words[:-1]
                        else:
                            answer_words = word_tokenize(answer)

                        prev_context_words = word_tokenize( p["context"][0:answer_start ] )
                        left_context_words = word_tokenize( p["context"][answer_start:] )
                        answer_reproduce = []
                        for i in range(len(answer_words)):
                            if i < len(left_context_words):
                                w = left_context_words[i]
                                answer_reproduce.append(w)
                        join_a = ' '.join(answer_words)
                        join_ar = ' '.join(answer_reproduce)

                        #if not ((join_ar in join_a) or (join_a in join_ar)):
                        if join_a != join_ar:
                            count += 1
                        
                        fpw.write(' '.join(prev_context_words+left_context_words)+'\t')
                        fpw.write(' '.join(question)+'\t')
                        pos_list = []
                        for i in range(len(answer_words)):
                            if i < len(left_context_words):
                                pos_list.append(str(len(prev_context_words)+i+1))
                        if len(pos_list) == 0:
                            print (join_ar)
                            print (join_a)
                            print ('answer:'+answer)
                        assert(len(pos_list) > 0)
                        fpw.write(' '.join(pos_list)+'\n')
                    else:
                        fpw.write(' '.join(word_tokenize( p["context"]) )+'\t')
                        fpw.write(' '.join(question)+'\n')

        fpw.close()
    print ('SQuAD preprossing finished!')

In [65]:
prepSQuAD()

SQuAD preprossing finished!
